In [1]:
from skimage import io, data

#myImage=io.imread("Documents/Data/Confocal-Multiphoton/MyProject/myConfocalImage.tif")
myImage=data.cells3d()

In [2]:
myImage.shape

(60, 2, 256, 256)

In [3]:
import napari
viewer = napari.Viewer()
viewer.add_image(myImage[:,1,:,:], rgb=False, name='Ch_1', opacity=0.8, rendering='mip', blending='additive', colormap='green')

<Image layer 'Ch_1' at 0x24d5a148d00>

In [4]:
th_ch=10000
myImage_ch=myImage[:,1,:,:]
binary_ch=myImage_ch>th_ch
viewer.add_image(binary_ch, rgb=False, name='threshold_'+str(th_ch), opacity=1.0, rendering='mip', blending='additive', colormap='gray')


<Image layer 'threshold_10000' at 0x24d69da0ca0>

In [5]:
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
import numpy as np

#Distance map
distance = ndi.distance_transform_edt(myImage_ch)
coords = peak_local_max(distance, footprint=np.ones((3, 3, 3)), labels=binary_ch, min_distance=5)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)   
#Watershed
labels = watershed(-distance, markers, mask=binary_ch)
viewer.add_image(labels, rgb=False, name='Labels', opacity=0.6, rendering='mip', blending='additive', colormap='inferno')

<Image layer 'Labels' at 0x24d6c2fbee0>

In [6]:
### Improving the precision in the labelling
from skimage import filters
myImage_ch_gf=filters.gaussian(myImage_ch, sigma=1)
viewer.add_image(myImage_ch_gf, rgb=False, name='GB', opacity=0.6, rendering='mip', blending='additive', colormap='gray')

<Image layer 'GB' at 0x24d6c3e9190>

In [7]:
from skimage import morphology
binary_ch=myImage_ch_gf>0.1
binary_ch=morphology.remove_small_objects(binary_ch, 7, connectivity=2)
#Distance map
distance = ndi.distance_transform_edt(myImage_ch)
coords = peak_local_max(distance, footprint=np.ones((20, 20, 10)), labels=binary_ch, min_distance=20)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)   
#Watershed
labels = watershed(-distance, markers, mask=binary_ch)
viewer.add_image(labels, rgb=False, name='Labels', opacity=0.6, rendering='mip', blending='additive', colormap='inferno')

<Image layer 'Labels [1]' at 0x24d6c3635e0>

In [8]:
points = np.array(coords)
viewer.add_points(points, name='Maxima', size=3)

<Points layer 'Maxima' at 0x24d6e4200a0>

In [10]:
from skimage import measure
import pandas as pd

prop_Ch1=measure.regionprops_table(labels, myImage[:,1,:,:],properties=['area','label','equivalent_diameter','mean_intensity','centroid','weighted_centroid'])
properties_Ch1={'Area': prop_Ch1['area'], 'Label': prop_Ch1['label'], 'Equivalent_diameter': prop_Ch1['equivalent_diameter'],
          'Centroid_X': prop_Ch1['centroid-0'], 'Centroid_Y': prop_Ch1['centroid-1'], 'Centroid_Z': prop_Ch1['centroid-2'],
          'Mean_intensity' : prop_Ch1['mean_intensity'],          
         }
properties_df=pd.DataFrame(properties_Ch1)
properties_df.to_csv('Documents/Data/Confocal-Multiphoton/MyProject/measured_properties.csv')
io.imsave('Documents/Data/Confocal-Multiphoton/MyProject/Labels.tif', labels)


C:\Users\dantuor\AppData\Local\Temp\ipykernel_5092\1166991390.py:11: UserWarning: Documents/Data/Confocal-Multiphoton/MyProject/Labels.tif is a low contrast image
  io.imsave('Documents/Data/Confocal-Multiphoton/MyProject/Labels.tif', labels)
